# Trucks detection with COLAB and Neptune

trying to follow https://neptune.ai/blog/how-to-train-your-own-object-detector-using-tensorflow-object-detection-api with my data


## Unzip images and annotations

In [1]:
! aws s3 sync s3://sagemaker-s3-luk/avocado/ images

download: s3://sagemaker-s3-luk/avocado/labels_avocadoproject_2021-10-10-09-33-06.zip to images/labels_avocadoproject_2021-10-10-09-33-06.zip
download: s3://sagemaker-s3-luk/avocado/iCloud Photos (5).zip to images/iCloud Photos (5).zip


In [56]:
%%bash
cd images
rm -rf 'iCloud Photos'
unzip -q 'iCloud Photos (5).zip'
rm -rf annotations
unzip -q labels_avocadoproject_2021-10-10-09-33-06.zip -d annotations
rm -rf avocadoproject
mkdir avocadoproject
mv annotations/* avocadoproject
mv 'iCloud Photos'/* avocadoproject
ls

annotations
avocadoproject
iCloud Photos
iCloud Photos (5).zip
img_and_annotations
labels_avocadoproject_2021-10-10-09-33-06.zip


In [59]:
# rm -r images/img_and_annotations

## Imports

In [13]:
!rm -rf models

In [14]:
# already done while creating a notebook

import os
import pathlib

# Clone the tensorflow models repository if it doesn't already exist
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 2935, done.
remote: Counting objects: 100% (2935/2935), done.
remote: Compressing objects: 100% (2472/2472), done.
remote: Total 2935 (delta 744), reused 1257 (delta 426), pack-reused 0
Receiving objects: 100% (2935/2935), 33.02 MiB | 15.72 MiB/s, done.
Resolving deltas: 100% (744/744), done.


In [2]:
%cd /home/ec2-user/SageMaker/models/research/

/home/ec2-user/SageMaker/models/research


In [16]:
# TODO: do it if not already prepared..
! protoc object_detection/protos/*.proto --python_out=/home/ec2-user/SageMaker/models/research

In [17]:
! cp object_detection/packages/tf2/setup.py .

In [3]:
!python -m pip install --use-deprecated=legacy-resolver . 
# !python -m pip install --use-feature=2020-resolver .

Processing /home/ec2-user/SageMaker/models/research
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 9.8 MB 29.0 MB/s eta 0:00:01
     |████████████████████████████████| 352 kB 58.9 MB/s eta 0:00:01
     |████████████████████████████████| 1.8 MB 57.1 MB/s eta 0:00:01
     |████████████████████████████████| 233 kB 66.1 MB/s eta 0:00:01
     |████████████████████████████████| 2.3 MB 51.6 MB/s eta 0:00:01
     |████████████████████████████████| 523 kB 54.3 MB/s eta 0:00:01
     |████████████████████████████████| 95 kB 846 kB/s  eta 0:00:01
     |████████████████████████████████| 21.9 MB 51.7 MB/s eta 0:00:01


     |████████████████████████████████| 1.2 MB 50.8 MB/s eta 0:00:01
     |████████████████████████████████| 90 kB 2.4 MB/s  eta 0:00:01
     |████████████████████████████████| 458.3 MB 16 kB/s s eta 0:00:01     |███████████████████████████████ | 443.3 MB 73.1 MB/s eta 0:00:01
     |████████████████████████████████| 108 kB 68.2 MB/s eta 0:00:01
     |████████████████████████████████| 213 kB 68.4 MB/s eta 0:00:01
     |████████████████████████████████| 4.0 MB 47.5 MB/s eta 0:00:01
     |████████████████████████████████| 43 kB 2.8 MB/s  eta 0:00:01
     |████████████████████████████████| 1.1 MB 42.9 MB/s eta 0:00:01
     |████████████████████████████████| 155 kB 74.2 MB/s eta 0:00:01
     |████████████████████████████████| 95 kB 5.4 MB/s  eta 0:00:01
     |████████████████████████████████| 154 kB 67.0 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 5.8 MB/s  eta 0:00:01
     |████████████████████████████████| 1.3 MB 36.6 MB/s eta 0:00:01
     |█████████████████████████████

     |████████████████████████████████| 781 kB 48.3 MB/s eta 0:00:01
     |████████████████████████████████| 4.9 MB 71.7 MB/s eta 0:00:01
     |████████████████████████████████| 146 kB 72.8 MB/s eta 0:00:01
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1668658 sha256=1d5a66f0e5b9faccac66f608eedb7ef66667814333641db99a274ba2d70a48ef
  Stored in directory: /tmp/pip-ephem-wheel-cache-k8ya_q1m/wheels/0f/f9/3c/80b1718c9d2eb5b6a4cae1495bfab9871f89c411c41b586424
  Created wheel for avro-python3: filename=avro_python3-1.10.2-py3-none-any.whl size=44009 sha256=b680e97dbd5918ad41c78551f337183fbcc5e396f5829c70c05421feace52683
  Stored in directory: /home/ec2-user/.cache/pip/wheels/5b/6f/ca/80164a844e4691238e34a8b139b4695354df4490db91be61f7
  Created wheel for pycocotools: filename=pycocotools-2.0.2-cp36-cp36m-linux_x86_64.whl size=301742 sha256=239fa514dc0fcc00addb7700bda0a7d174f8f0e8340c90797102fd35a40e9003
  Stored in directory: /home/ec2-user/.cache/p

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [4]:
%cd /home/ec2-user/SageMaker

/home/ec2-user/SageMaker


In [5]:
import matplotlib
import matplotlib.pyplot as plt

import os
import random
import io
import imageio
import glob
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display, Javascript
from IPython.display import Image as IPyImage

import tensorflow as tf

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

from object_detection.utils import dataset_util
from lxml import etree
import PIL.Image
import tensorflow.compat.v1 as compat_tf
import hashlib

import glob

%matplotlib inline

# Utilities

# Avocado dataset

In [81]:
# TODO: remove orphans jpegs which does not have xml

In [ ]:
! rm images/avocadoproject

In [83]:
%rm "images/avocadoproject/IMG_1078.JPEG" 

In [78]:
%%bash
for filename in images/avocadoproject/*.xml
do
    sed -i 's/Unspecified/0/g' $filename
done

In [69]:
IMAGES_AND_LABELS_FOLDER = "images/avocadoproject"

In [84]:
import random
import math
import os

def train_valid_test_split(path, split_ratio=(0.8,0.1,0.1)):
  all_images = list(os.path.basename(filename) for filename in glob.glob(os.path.join(path, "*.JPEG")))
  if len(split_ratio) != 3:
    raise AttributeError("you should provide a tuple with 3 fractions for split- train,valid,test")
  if sum(split_ratio) != 1:
    raise AttributeError("Split should add up to 1.0")
  train_len = math.floor(split_ratio[0] * len(all_images))
  random.seed(10)
  train_images = random.sample(all_images, train_len)
  other_images = list(set(all_images) - set(train_images))
  valid_len = math.floor(split_ratio[1] * len(all_images))
  valid_images = random.sample(other_images, valid_len)
  test_images = list(set(other_images) - set(valid_images))
  return train_images, valid_images, test_images

train_images, valid_images, test_images = train_valid_test_split(IMAGES_AND_LABELS_FOLDER)

print("train has %d elements, valid %d, test %d" % (len(train_images), len(valid_images), len(test_images)))
print(train_images[:5], valid_images[:5])

train has 119 elements, valid 14, test 16
['IMG_1218.JPEG', 'IMG_1109.JPEG', 'IMG_1108.JPEG', 'IMG_1189.JPEG', 'IMG_1209.JPEG'] ['IMG_1133.JPEG', 'IMG_1111.JPEG', 'IMG_1081.JPEG', 'IMG_1110.JPEG', 'IMG_1096.JPEG']


In [64]:
print(test_images)

['IMG_1096.JPEG', 'IMG_1196.JPEG', 'IMG_1165.JPEG', 'IMG_1218.JPEG', 'IMG_1116.JPEG', 'IMG_1179.JPEG', 'IMG_1210.JPEG', 'IMG_1085.JPEG', 'IMG_1101.JPEG', 'IMG_1208.JPEG', 'IMG_1182.JPEG', 'IMG_1078.JPEG', 'IMG_1123.JPEG', 'IMG_1113.JPEG', 'IMG_1192.JPEG']


In [103]:
!mkdir images/test_images

In [105]:
source_folder = 'images/avocadoproject/'
dest_folder = 'images/test_images/'

import shutil

for test_image in test_images:
    shutil.copy(source_folder + test_image, dest_folder + test_image)

In [106]:
%ls images/test_images

IMG_1085.JPEG  IMG_1132.JPEG  IMG_1165.JPEG  IMG_1210.JPEG
IMG_1088.JPEG  IMG_1137.JPEG  IMG_1167.JPEG  IMG_1220.JPEG
IMG_1114.JPEG  IMG_1150.JPEG  IMG_1182.JPEG  IMG_1223.JPEG
IMG_1130.JPEG  IMG_1161.JPEG  IMG_1206.JPEG  IMG_1224.JPEG


## Create TFRecords

In [71]:
from object_detection.utils import label_map_util

In [49]:
! echo "item {\
  id: 1\
  name: 'avocado'\
}" > /tmp/avocado_labels.pbtxt

In [95]:
label_map_path = "/tmp/avocado_labels.pbtxt"
label_map_util.get_label_map_dict(label_map_path)

{'avocado': 1}

In [85]:
# https://github.com/tensorflow/models/blob/master/research/object_detection/dataset_tools/create_pascal_tf_record.py 
# customized for my needs

annotations_dir = IMAGES_AND_LABELS_FOLDER
data_dir = "images"
tf_record_folder = "./tfrecord"
if not os.path.exists(tf_record_folder):
  os.mkdir(tf_record_folder)



def dict_to_tf_example(data,
                       dataset_directory,
                       label_map_dict,
                       ignore_difficult_instances=False):
  """Convert XML derived dict to tf.Example proto.
  Notice that this function normalizes the bounding box coordinates provided
  by the raw data.
  Args:
    data: dict holding PASCAL XML fields for a single image (obtained by
      running dataset_util.recursive_parse_xml_to_dict)
    dataset_directory: Path to root directory holding PASCAL dataset
    label_map_dict: A map from string label names to integers ids.
    ignore_difficult_instances: Whether to skip difficult instances in the
      dataset  (default: False).
    image_subdirectory: String specifying subdirectory within the
      PASCAL dataset directory holding the actual image data.
  Returns:
    example: The converted tf.Example.
  Raises:
    ValueError: if the image pointed to by data['filename'] is not a valid JPEG
  """
  img_path = os.path.join(data['folder'], data['filename'])
  full_path = os.path.join(dataset_directory, img_path)
  with compat_tf.gfile.GFile(full_path, 'rb') as fid:
    encoded_jpg = fid.read()
  encoded_jpg_io = io.BytesIO(encoded_jpg)
  image = PIL.Image.open(encoded_jpg_io)
  if image.format != 'JPEG':
    raise ValueError('Image format not JPEG')
  key = hashlib.sha256(encoded_jpg).hexdigest()

  width = int(data['size']['width'])
  height = int(data['size']['height'])

  xmin = []
  ymin = []
  xmax = []
  ymax = []
  classes = []
  classes_text = []
  truncated = []
  poses = []
  difficult_obj = []
  if 'object' in data:
    for obj in data['object']:
      difficult = bool(int(obj['difficult']))
      if ignore_difficult_instances and difficult:
        continue

      difficult_obj.append(int(difficult))

      xmin.append(float(obj['bndbox']['xmin']) / width)
      ymin.append(float(obj['bndbox']['ymin']) / height)
      xmax.append(float(obj['bndbox']['xmax']) / width)
      ymax.append(float(obj['bndbox']['ymax']) / height)
      classes_text.append(obj['name'].encode('utf8'))
      classes.append(label_map_dict[obj['name']])
      truncated.append(int(obj['truncated']))
      poses.append(obj['pose'].encode('utf8'))

  example = compat_tf.train.Example(features=compat_tf.train.Features(feature={
      'image/height': dataset_util.int64_feature(height),
      'image/width': dataset_util.int64_feature(width),
      'image/filename': dataset_util.bytes_feature(
          data['filename'].encode('utf8')),
      'image/source_id': dataset_util.bytes_feature(
          data['filename'].encode('utf8')),
      'image/key/sha256': dataset_util.bytes_feature(key.encode('utf8')),
      'image/encoded': dataset_util.bytes_feature(encoded_jpg),
      'image/format': dataset_util.bytes_feature('jpeg'.encode('utf8')),
      'image/object/bbox/xmin': dataset_util.float_list_feature(xmin),
      'image/object/bbox/xmax': dataset_util.float_list_feature(xmax),
      'image/object/bbox/ymin': dataset_util.float_list_feature(ymin),
      'image/object/bbox/ymax': dataset_util.float_list_feature(ymax),
      'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
      'image/object/class/label': dataset_util.int64_list_feature(classes),
      'image/object/difficult': dataset_util.int64_list_feature(difficult_obj),
      'image/object/truncated': dataset_util.int64_list_feature(truncated),
      'image/object/view': dataset_util.bytes_list_feature(poses),
  }))
  return example



def files_to_tfrecord(filenames, data_dir, output_path, ignore_difficult_instances=False):
  filenames_no_extensions = [os.path.splitext(fn)[0] for fn in filenames]
  label_map_dict = label_map_util.get_label_map_dict(label_map_path)
  examples = []
  with compat_tf.python_io.TFRecordWriter(output_path) as writer:
    for idx, example in enumerate(filenames_no_extensions):
      path = os.path.join(annotations_dir, example + '.xml')
      with compat_tf.gfile.GFile(path, 'r') as fid:
        xml_str = fid.read()    
      xml = etree.fromstring(xml_str)
      data = dataset_util.recursive_parse_xml_to_dict(xml)['annotation']
      tf_example = dict_to_tf_example(data, data_dir, label_map_dict,
                                      ignore_difficult_instances)
#       print(tf_example)
#       break
      examples.append(tf_example)
      writer.write(tf_example.SerializeToString())
  # return examples

files_to_tfrecord(train_images, data_dir, os.path.join(tf_record_folder, 'train'))
files_to_tfrecord(valid_images, data_dir, os.path.join(tf_record_folder, 'valid'))
files_to_tfrecord(test_images, data_dir, os.path.join(tf_record_folder, 'test'))

In [86]:
label_map_util.get_label_map_dict(label_map_path)

{'avocado': 1}

In [90]:
!cp /tmp/avocado_labels.pbtxt ./tfrecord

In [92]:
!aws s3 sync ./tfrecord s3://sagemaker-s3-luk/trucks_tfrecord

## Pipeline

In [6]:
# model_name = "efficientdet_d1_coco17_tpu-32"
# model_name = "ssd_resnet50_v1_fpn_640x640_coco17_tpu-8"
# model_name = "ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8"
# model_name = "ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8"
# model_name= "ssd_mobilenet_v2_320x320_coco17_tpu-8"
# model_name = "ssd_resnet50_v1_fpn_640x640_coco17_tpu-8"
model_name = "ssd_inception_v2_coco_2018_01_28"

original_pipeline_path = f"pretrained_models/{model_name}/pipeline.config"
pipeline_path = f"pretrained_models/{model_name}/pipeline.docker.confignew"
# checkpoint_path = f"workspace/pre_trained_models/{model_name}/checkpoint/ckpt-0"
checkpoint_path = "checkpoint/ckpt-0"
# checkpoint_path = "/content/workspace/model_customized/saved_checkpoint/ckpt-4"

from google.protobuf import text_format
from object_detection.protos import pipeline_pb2

docker_data_folder = "/opt/ml/input/data/train/"
docker_map_path = docker_data_folder + "avocado_labels.pbtxt"


def edit_pipeline_config(old_path, new_path, cfg):
  pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()                                                                                                                                                                                                          

  with compat_tf.gfile.GFile(old_path, "r") as f:                                                                                                                                                                                                                     
      proto_str = f.read()                                                                                                                                                                                                                                          
      text_format.Merge(proto_str, pipeline_config)                                                                                                                                                                                                                 

  pipeline_config.model.ssd.num_classes = cfg['num_classes']
  pipeline_config.train_config.batch_size = cfg['batch_size']
  pipeline_config.train_config.fine_tune_checkpoint = cfg['fine_tune_checkpoint']
  pipeline_config.train_config.fine_tune_checkpoint_type = 'detection'
  pipeline_config.train_input_reader.label_map_path = cfg['label_map_path']
  pipeline_config.train_input_reader.tf_record_input_reader.input_path[0] = cfg['train_tf_path']
  pipeline_config.eval_input_reader[0].label_map_path = cfg['label_map_path']
  pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[0] = cfg['eval_tf_path']
  # print(pipeline_config)


  config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
  with compat_tf.gfile.Open(new_path, "wb") as f:                                                                                                                                                                                                                       
      f.write(config_text)                                                                                                                                                                                                                                          

new_config = {
    "batch_size": 8,
    "num_classes": 1,
    "label_map_path": docker_map_path,
    "train_tf_path": os.path.join(docker_data_folder, 'train'),
    "eval_tf_path": os.path.join(docker_data_folder, 'valid'),
    "fine_tune_checkpoint": checkpoint_path
}

edit_pipeline_config(original_pipeline_path, pipeline_path, new_config)

pipeline_path

'pretrained_models/ssd_inception_v2_coco_2018_01_28/pipeline.docker.confignew'

In [7]:
! cat 'pretrained_models/ssd_inception_v2_coco_2018_01_28/pipeline.docker.confignew'

model {
  ssd {
    num_classes: 1
    image_resizer {
      fixed_shape_resizer {
        height: 300
        width: 300
      }
    }
    feature_extractor {
      type: "ssd_inception_v2"
      depth_multiplier: 1.0
      min_depth: 16
      conv_hyperparams {
        regularizer {
          l2_regularizer {
            weight: 4e-05
          }
        }
        initializer {
          truncated_normal_initializer {
            mean: 0.0
            stddev: 0.03
          }
        }
        activation: RELU_6
        batch_norm {
          decay: 0.9997
          center: true
          scale: true
          epsilon: 0.001
          train: true
        }
      }
    }
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_u